In [3]:
#pip install pyttsx3

In [2]:
from bs4 import BeautifulSoup

In [3]:
#!pip install requests

In [4]:
import requests

In [126]:
url="http://164.100.47.194/Loksabha/Members/memberwisetotal.aspx"
page= requests.get(url)

In [6]:
page

<Response [200]>

In [ ]:
#200 represents the page is loaded successfully

# Analyzing 'td' and 'tr' tags

In [7]:
soup=BeautifulSoup(page.content, 'html.parser')

In [111]:
tr_tags=soup.find("table", id="ContentPlaceHolder1_tblMember", width="100%", border="0", cellspacing="0", cellpadding="0").find_all('tr')
print(len(tr_tags))

39


In [9]:
#39 tr's are there
#we have 35 Membernames on the page 1.

In [11]:
td_tags=soup.find("table", id="ContentPlaceHolder1_tblMember", width="100%", border="0", cellspacing="0", cellpadding="0").find_all('td')
td_tags=td_tags[2:]
td_tags

[<td align="center" bgcolor="#E09819" valign="top" width="10%"><b>Division NO</b></td>,
 <td align="left" bgcolor="#E09819" valign="top" width="40%"><b>Member Name</b></td>,
 <td align="center" bgcolor="#E09819" valign="top" width="25%"><b>No. of days Members signed the Register</b></td>,
 <td align="center" bgcolor="#E09819" valign="top" width="25%"><b>No. of days Members not signed the Register</b></td>,
 <td align="center">5</td>,
 <td align="center">
 <a href="mpwiseresults.aspx?lsno=17&amp;sesno=9&amp;mpsno=3979">
                                             Shri  D.V. Sadananda  Gowda                                             </a>
 </td>,
 <td align="center"><font size="3">4</font></td>,
 <td align="center"><font size="3">0</font></td>,
 <td align="center">9</td>,
 <td align="center">
 <a href="mpwiseresults.aspx?lsno=17&amp;sesno=9&amp;mpsno=4776">
                                             Dr.  Harsh  Vardhan                                           </a>
 </td>,
 <td align

In [212]:
tr_tags=soup.find("table", id="ContentPlaceHolder1_tblMember", width="100%", border="0", cellspacing="0", cellpadding="0").find_all('tr')
#tr_tags=tr_tags[2:]
#tr_tags=tr_tags[:-1]
tr_tags

[<tr>
 <td colspan="2">
 <div>
 <table border="1" bordercolor="#8080FF" cellpadding="1" cellspacing="1" id="ContentPlaceHolder1_DataGrid1" rules="all" width="100%">
 <tr class="paging">
 <td colspan="4"><span>1</span> <a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$DataGrid1$ctl01$ctl01','')">2</a> <a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$DataGrid1$ctl01$ctl02','')">3</a> <a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$DataGrid1$ctl01$ctl03','')">4</a> <a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$DataGrid1$ctl01$ctl04','')">5</a> <a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$DataGrid1$ctl01$ctl05','')">6</a> <a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$DataGrid1$ctl01$ctl06','')">7</a> <a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$DataGrid1$ctl01$ctl07','')">8</a> <a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$DataGrid1$ctl01$ctl08','')">9</a> <a href="javascript:__doPostBa

# FULL CODE::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::


In [604]:
#FULL CODE::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

url="http://164.100.47.194/Loksabha/Members/memberwisetotal.aspx"
page= requests.get(url)


source = requests.get(url)
source.raise_for_status()

soup=BeautifulSoup(source.text, 'html.parser')

tr_tags=soup.find("table", id="ContentPlaceHolder1_tblMember", width="100%", border="0", cellspacing="0", cellpadding="0").find_all('tr')
tr_tags=tr_tags[2:]
tr_tags=tr_tags[:-1]
tr_tags

df2=[]
for tr in tr_tags:
    td = tr.find_all('td')
    row = [i.text.replace('\n','').replace('\r','').strip() for i in td] 
    df2.append(row)
    #print(row)
    
df=pd.DataFrame(df2)
df. columns=df. iloc[0]
df=df[1:]
df.reset_index(inplace=True, drop=True)


#Capturing Links on Member Names:::::
tr_tags=soup.find("table", id="ContentPlaceHolder1_tblMember", width="100%", border="0", cellspacing="0", cellpadding="0").find_all('tr')
i=[]
for tr in tr_tags:
   
    link_tag= tr.find('a')
    link= link_tag.get('href') if link_tag else 'N/A'
    i.append(link)
    #print(link)

    
df2= pd.DataFrame(i)
df2=df2[3:-1]
df2.rename(columns = {0:'Member link'}, inplace = True)
df2.reset_index(inplace=True)
df2['Member link'] = df2['Member link'].apply(lambda x: f"http://164.100.47.194/Loksabha/Members/{x}")

#Concatenating df and df2
df=pd.concat([df, df2], axis=1).drop('index', axis=1)


#ACCESSING MEMBER LINK URLS::
member_attendance_urls=df['Member link']
urls=member_attendance_urls

df3=[]

for  url in urls:
    source = requests.get(url)
    source.raise_for_status()

    soup=BeautifulSoup(source.text, 'html.parser')

    tr_tags=soup.find("table", id="ContentPlaceHolder1_tblMember", width="100%", border="0", cellspacing="0", cellpadding="0").find_all('tr')
    tr_tags

    for tr in tr_tags:
        td = tr.find_all('td')
        row = [i.text for i in td]
        df3.append(row)
        #print(row)

df1=pd.DataFrame(df3)
df1=df1[2:-1][[0,1]]
df1.columns= df1.iloc[0]
df1=df1[1:]
df1.reset_index(inplace=True, drop=True)
df1['Date of Attendance'] = pd.to_datetime(df1['Date of Attendance'], errors='coerce')
df1 = df1.dropna(subset=['Date of Attendance'])
df1=df1.reset_index(drop=True)

#Converting Division NO to numeric in order to sort values well.
df['Division NO'] = pd.to_numeric(df['Division NO'])

#Duplicating records for df dataframe to "x" times to match the length of df1 dataframe!
x=int(len(df1)/len(df))

df_duplicated=pd.concat([df]*x, ignore_index=True).sort_values('Division NO')

df_duplicated=df_duplicated.reset_index(drop=True)

#Joining df1 and df_duplicated on the basis of index!
df_final=df_duplicated.join(df1).drop('Member link', axis=1)
df_final.head(30)

,Division NO,Member Name,No. of days Members signed the Register,No. of days Members not signed the Register,Date of Attendance,Attendance Status
0,5,Shri D.V. Sadananda Gowda,4,1,2022-07-18,S
1,5,Shri D.V. Sadananda Gowda,4,1,2022-07-19,S
2,5,Shri D.V. Sadananda Gowda,4,1,2022-07-20,S
3,5,Shri D.V. Sadananda Gowda,4,1,2022-07-21,S
4,5,Shri D.V. Sadananda Gowda,4,1,2022-07-22,NS
5,9,Dr. Harsh Vardhan,5,0,2022-07-18,S
6,9,Dr. Harsh Vardhan,5,0,2022-07-19,S
7,9,Dr. Harsh Vardhan,5,0,2022-07-20,S
8,9,Dr. Harsh Vardhan,5,0,2022-07-21,S
9,9,Dr. Harsh Vardhan,5,0,2022-07-22,S


# Stepwise Code:::

# Dataframe Making::

In [535]:
url="http://164.100.47.194/Loksabha/Members/memberwisetotal.aspx"
page= requests.get(url)


source = requests.get(url)
source.raise_for_status()

soup=BeautifulSoup(source.text, 'html.parser')

tr_tags=soup.find("table", id="ContentPlaceHolder1_tblMember", width="100%", border="0", cellspacing="0", cellpadding="0").find_all('tr')
tr_tags=tr_tags[2:]
tr_tags=tr_tags[:-1]
tr_tags

df2=[]
for tr in tr_tags:
    td = tr.find_all('td')
    row = [i.text.replace('\n','').replace('\r','').strip() for i in td] 
    df2.append(row)
    #print(row)
    
df=pd.DataFrame(df2)
df. columns=df. iloc[0]
df=df[1:]
df.reset_index(inplace=True, drop=True)
df

,Division NO,Member Name,No. of days Members signed the Register,No. of days Members not signed the Register
0,5,Shri D.V. Sadananda Gowda,4,1
1,9,Dr. Harsh Vardhan,5,0
2,12,Shri Santosh Kumar Gangwar,5,0
3,15,Dr. Jitendra Singh,5,0
4,22,Dr. Sanjay Jaiswal,4,1
5,23,Shri Ratan Lal Kataria,5,0
6,31,Shri Rakesh Singh,3,2
7,32,Shri Nishikant Dubey,5,0
8,33,Shri Ganesh Singh,3,2
9,34,Dr. Kirit Solanki,5,0


In [536]:
#Capturing Links on Member Names:::::
tr_tags=soup.find("table", id="ContentPlaceHolder1_tblMember", width="100%", border="0", cellspacing="0", cellpadding="0").find_all('tr')
i=[]
for tr in tr_tags:
   
    link_tag= tr.find('a')
    link= link_tag.get('href') if link_tag else 'N/A'
    i.append(link)
    #print(link)

    
df2= pd.DataFrame(i)
df2=df2[3:-1]
df2.rename(columns = {0:'Member link'}, inplace = True)
df2.reset_index(inplace=True)
df2['Member link'] = df2['Member link'].apply(lambda x: f"http://164.100.47.194/Loksabha/Members/{x}")

#Concatenating df and df2
df=pd.concat([df, df2], axis=1).drop('index', axis=1)
df

,Division NO,Member Name,No. of days Members signed the Register,No. of days Members not signed the Register,Member link
0,5,Shri D.V. Sadananda Gowda,4,1,http://164.100.47.194/Loksabha/Members/mpwiser...
1,9,Dr. Harsh Vardhan,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...
2,12,Shri Santosh Kumar Gangwar,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...
3,15,Dr. Jitendra Singh,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...
4,22,Dr. Sanjay Jaiswal,4,1,http://164.100.47.194/Loksabha/Members/mpwiser...
5,23,Shri Ratan Lal Kataria,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...
6,31,Shri Rakesh Singh,3,2,http://164.100.47.194/Loksabha/Members/mpwiser...
7,32,Shri Nishikant Dubey,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...
8,33,Shri Ganesh Singh,3,2,http://164.100.47.194/Loksabha/Members/mpwiser...
9,34,Dr. Kirit Solanki,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...


# Accessing Member Links::::

In [580]:
member_attendance_urls=df['Member link']
urls=member_attendance_urls

df3=[]

for  url in urls:
    source = requests.get(url)
    source.raise_for_status()

    soup=BeautifulSoup(source.text, 'html.parser')

    tr_tags=soup.find("table", id="ContentPlaceHolder1_tblMember", width="100%", border="0", cellspacing="0", cellpadding="0").find_all('tr')
    tr_tags

    for tr in tr_tags:
        td = tr.find_all('td')
        row = [i.text for i in td]
        df3.append(row)
        #print(row)

df1=pd.DataFrame(df3)
df1=df1[2:-1][[0,1]]
df1.columns= df1.iloc[0]
df1=df1[1:]
df1.reset_index(inplace=True, drop=True)
df1['Date of Attendance'] = pd.to_datetime(df1['Date of Attendance'], errors='coerce')
df1 = df1.dropna(subset=['Date of Attendance'])
df1=df1.reset_index(drop=True)
df1

2,Date of Attendance,Attendance Status
0,2022-07-18,S
1,2022-07-19,S
2,2022-07-20,S
3,2022-07-21,S
4,2022-07-22,NS
...,...,...
170,2022-07-18,S
171,2022-07-19,S
172,2022-07-20,S
173,2022-07-21,S


Next task: merge df and df1 to create a final expanded datframe!

In [581]:
len(df1)

175

In [582]:
len(df)

35

In [583]:
int(len(df1)/len(df))

5

In [584]:
#Converting Division NO to numeric in order to sort values well.
df['Division NO'] = pd.to_numeric(df['Division NO'])

#Duplicating records for df dataframe to x times to match the length of df1 dataframe!
x=int(len(df1)/len(df))

df_duplicated=pd.concat([df]*x, ignore_index=True).sort_values('Division NO')

df_duplicated=df_duplicated.reset_index(drop=True)

In [585]:
len(df_duplicated)

175

In [586]:
#Now we need to merge df1 and df_duplicated on the basis of index!

In [587]:
df_duplicated

,Division NO,Member Name,No. of days Members signed the Register,No. of days Members not signed the Register,Member link
0,5,Shri D.V. Sadananda Gowda,4,1,http://164.100.47.194/Loksabha/Members/mpwiser...
1,5,Shri D.V. Sadananda Gowda,4,1,http://164.100.47.194/Loksabha/Members/mpwiser...
2,5,Shri D.V. Sadananda Gowda,4,1,http://164.100.47.194/Loksabha/Members/mpwiser...
3,5,Shri D.V. Sadananda Gowda,4,1,http://164.100.47.194/Loksabha/Members/mpwiser...
4,5,Shri D.V. Sadananda Gowda,4,1,http://164.100.47.194/Loksabha/Members/mpwiser...
...,...,...,...,...,...
170,63,Shri Bidyut Mahato,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...
171,63,Shri Bidyut Mahato,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...
172,63,Shri Bidyut Mahato,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...
173,63,Shri Bidyut Mahato,5,0,http://164.100.47.194/Loksabha/Members/mpwiser...


In [593]:
#Joining df1 and df_duplicated on the basis of index!
df_final=df_duplicated.join(df1).drop('Member link', axis=1)
df_final

,Division NO,Member Name,No. of days Members signed the Register,No. of days Members not signed the Register,Date of Attendance,Attendance Status
0,5,Shri D.V. Sadananda Gowda,4,1,2022-07-18,S
1,5,Shri D.V. Sadananda Gowda,4,1,2022-07-19,S
2,5,Shri D.V. Sadananda Gowda,4,1,2022-07-20,S
3,5,Shri D.V. Sadananda Gowda,4,1,2022-07-21,S
4,5,Shri D.V. Sadananda Gowda,4,1,2022-07-22,NS
...,...,...,...,...,...,...
170,63,Shri Bidyut Mahato,5,0,2022-07-18,S
171,63,Shri Bidyut Mahato,5,0,2022-07-19,S
172,63,Shri Bidyut Mahato,5,0,2022-07-20,S
173,63,Shri Bidyut Mahato,5,0,2022-07-21,S


In [594]:
#FINAL DATAFRAME:: df_final

In [595]:
df_final.head(30)

,Division NO,Member Name,No. of days Members signed the Register,No. of days Members not signed the Register,Date of Attendance,Attendance Status
0,5,Shri D.V. Sadananda Gowda,4,1,2022-07-18,S
1,5,Shri D.V. Sadananda Gowda,4,1,2022-07-19,S
2,5,Shri D.V. Sadananda Gowda,4,1,2022-07-20,S
3,5,Shri D.V. Sadananda Gowda,4,1,2022-07-21,S
4,5,Shri D.V. Sadananda Gowda,4,1,2022-07-22,NS
5,9,Dr. Harsh Vardhan,5,0,2022-07-18,S
6,9,Dr. Harsh Vardhan,5,0,2022-07-19,S
7,9,Dr. Harsh Vardhan,5,0,2022-07-20,S
8,9,Dr. Harsh Vardhan,5,0,2022-07-21,S
9,9,Dr. Harsh Vardhan,5,0,2022-07-22,S
